In [486]:
import numpy as np
from enum import Enum
from collections import defaultdict 

In [487]:
class FuzzySet:

#Static Variables
    y_trig = (0,1,0)
    y_trapz = (0,1,1,0)
    Type = Enum('Fuzzy Set Type','trig trapz')

    @staticmethod
    def pair_cross_diff(multiple,x,y,x_len) :
        multiple_diff = 0
        for i in range(x_len - 1):
            multiple_diff += (multiple[i] + multiple[i + 1]) * (x[i] * y[i + 1] - x[i + 1] * y[i])
        return multiple_diff

#Instance Variables:
#   x, x_len, y, type, ys, centriod
    def get_x(self):        return self.x
    def get_x_len(self):    return self.x_len
    def get_ys(self):       return self.ys

    def __init__(self,x):
        self.x = x
        self.x_len = len(x)
        if self.x_len == 3:
            self.type = FuzzySet.Type.trig.value
        else:
            self.type = FuzzySet.Type.trapz.value
        self.ys()
        self.centriod()
        return
    
    def __str__(self):
        return 'Type : %s\nVertices Number : %s\nX Vertices : %s\nY Vertices : %s\nLine Equations :  %s\nThere Intervals : %s\nCentriod : %s' % (self.type, self.x_len, self.x, self.y, self.ys,self.intervals,self.centriod)

    def __repr__(self):
        return '\n%s\n' % (self.__str__())
    
    def ys(self):
        self.ys = []
        self.intervals = []
        if self.type == FuzzySet.Type.trig.value:
            self.y = FuzzySet.y_trig;
        else:
            self.y = FuzzySet.y_trapz
         
        for i in range(self.x_len - 1):
            if self.x[i] != self.x[i+1]:
                m = (self.y[i+1] - self.y[i]) / (self.x[i+1] - self.x[i])
                c = self.y[i] - m * self.x[i]
                self.ys.append((m, c))
                self.intervals.append((self.x[i], self.x[i+1]))
                
        if len(self.ys) == 0:
            self.ys.append((0, 1))
            self.intervals.append((self.x[0], self.x[0]))
        return
    
    def centriod(self):
        if self.x_len == 3:
            self.centriod = (self.x[0] + self.x[1] + self.x[2]) / 3
        else:
            signed_area = 0.5 * FuzzySet.pair_cross_diff([0.5] * self.x_len,self.x,self.y,self.x_len)
            self.centriod = 1 / (6 * signed_area) * FuzzySet.pair_cross_diff(self.x,self.x,self.y,self.x_len)
        return

    def mu(self,x):
        for i in range(len(self.intervals)):
            if x >= self.intervals[i][0] and x <= self.intervals[i][1]:
                return self.ys[i][0] * x + self.ys[i][1]
        return 0


In [488]:
soft_set = FuzzySet((0,0,20,40))
print(soft_set)

Type : 2
Vertices Number : 4
X Vertices : (0, 0, 20, 40)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.0, 1.0), (-0.05, 2.0)]
There Intervals : [(0, 20), (20, 40)]
Centriod : 15.555555555555555


In [489]:
ordinary_set = FuzzySet((20,40, 60, 80))
print(ordinary_set)

Type : 2
Vertices Number : 4
X Vertices : (20, 40, 60, 80)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -1.0), (0.0, 1.0), (-0.05, 4.0)]
There Intervals : [(20, 40), (40, 60), (60, 80)]
Centriod : 50.0


In [490]:
hard_set = FuzzySet((60, 80, 100, 100))
print(hard_set)

Type : 2
Vertices Number : 4
X Vertices : (60, 80, 100, 100)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -3.0), (0.0, 1.0)]
There Intervals : [(60, 80), (80, 100)]
Centriod : 84.44444444444444


In [491]:
class Variable:

    Type = Enum('Variable Type','input output')

#Static Variables
#Instance Variables:
#   sets, crisp_value

    def __init__(self):
        self.sets = {}
        self.crisp_value = None
        return
    
    def __str__(self):
        return 'Crisp Value : %s\nSets :%s' % (self.crisp_value, self.sets)
    
    def __repr__(self):
        return  '\n%s'% (self.__str__())

    def set_crisp(self,x):
        self.crisp_value = x
        return

    def update_set(self,name,x):
        self.sets[name] = FuzzySet(x)
        return

    def get_set_mu(self,name):
        return self.sets[name].mu(self.crisp_value)
    
    def get_set_centriod(self,name):
        return self.sets[name].centriod

    def get_likelhood_set(self,x):
        max_mu = -1
        max_set = None 
        for st in self.sets :
            mu = self.sets[st].mu(x)
            if mu >= max_mu :
                max_mu = mu
                max_set = st
        return max_set
            

In [ ]:
Unit of FUZZIFICATION

In [492]:
project_funding = Variable()
project_funding.update_set('very low',(0,0,10,30))
project_funding.update_set('low',(10,30,40,60))
project_funding.update_set('medium',(40,60,70,90))
project_funding.update_set('high',(70,90,100,100))
print(project_funding)
project_funding.set_crisp(50)
project_funding.get_set_mu('high')

Crisp Value : None
Sets :{'very low': 
Type : 2
Vertices Number : 4
X Vertices : (0, 0, 10, 30)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.0, 1.0), (-0.05, 1.5)]
There Intervals : [(0, 10), (10, 30)]
Centriod : 10.833333333333334
, 'low': 
Type : 2
Vertices Number : 4
X Vertices : (10, 30, 40, 60)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -0.5), (0.0, 1.0), (-0.05, 3.0)]
There Intervals : [(10, 30), (30, 40), (40, 60)]
Centriod : 35.0
, 'medium': 
Type : 2
Vertices Number : 4
X Vertices : (40, 60, 70, 90)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -2.0), (0.0, 1.0), (-0.05, 4.5)]
There Intervals : [(40, 60), (60, 70), (70, 90)]
Centriod : 65.0
, 'high': 
Type : 2
Vertices Number : 4
X Vertices : (70, 90, 100, 100)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -3.5), (0.0, 1.0)]
There Intervals : [(70, 90), (90, 100)]
Centriod : 89.16666666666667
}


0

In [493]:
team_experience_level = Variable()
team_experience_level.update_set('beginner',(0,15,30))
team_experience_level.update_set('intermediate',(15,30,45))
team_experience_level.update_set('expert',(30,60,60))
print(team_experience_level.get_likelhood_set(15),team_experience_level.get_likelhood_set(30),team_experience_level.get_likelhood_set(60),team_experience_level.get_likelhood_set(22))
print(team_experience_level)

beginner intermediate expert beginner
Crisp Value : None
Sets :{'beginner': 
Type : 1
Vertices Number : 3
X Vertices : (0, 15, 30)
Y Vertices : (0, 1, 0)
Line Equations :  [(0.06666666666666667, 0.0), (-0.06666666666666667, 2.0)]
There Intervals : [(0, 15), (15, 30)]
Centriod : 15.0
, 'intermediate': 
Type : 1
Vertices Number : 3
X Vertices : (15, 30, 45)
Y Vertices : (0, 1, 0)
Line Equations :  [(0.06666666666666667, -1.0), (-0.06666666666666667, 3.0)]
There Intervals : [(15, 30), (30, 45)]
Centriod : 30.0
, 'expert': 
Type : 1
Vertices Number : 3
X Vertices : (30, 60, 60)
Y Vertices : (0, 1, 0)
Line Equations :  [(0.03333333333333333, -1.0)]
There Intervals : [(30, 60)]
Centriod : 50.0
}


In [494]:
risk = Variable()
risk.update_set('low',(0,25,50))
risk.update_set('normal',(25,50,75))
risk.update_set('high',(50,100,100))
print(risk)

Crisp Value : None
Sets :{'low': 
Type : 1
Vertices Number : 3
X Vertices : (0, 25, 50)
Y Vertices : (0, 1, 0)
Line Equations :  [(0.04, 0.0), (-0.04, 2.0)]
There Intervals : [(0, 25), (25, 50)]
Centriod : 25.0
, 'normal': 
Type : 1
Vertices Number : 3
X Vertices : (25, 50, 75)
Y Vertices : (0, 1, 0)
Line Equations :  [(0.04, -1.0), (-0.04, 3.0)]
There Intervals : [(25, 50), (50, 75)]
Centriod : 50.0
, 'high': 
Type : 1
Vertices Number : 3
X Vertices : (50, 100, 100)
Y Vertices : (0, 1, 0)
Line Equations :  [(0.02, -1.0)]
There Intervals : [(50, 100)]
Centriod : 83.33333333333333
}


In [495]:
class Rule:
    
    @staticmethod
    def NOT(mu)      : return 1 - mu
    @staticmethod
    def OR(mu1,mu2)  : return max(mu1,mu2)
    @staticmethod
    def AND(mu1,mu2) : return min(mu1,mu2)
    @staticmethod
    def get_arg(arg) : return tuple(arg.split('='))
    @staticmethod
    def append_oprands(expr,and_oprand):
        for i in range(-1, -len(and_oprand)-1, -1) :
            if  and_oprand[i] !=  '^' :
                expr.append(Rule.get_arg(and_oprand[i]))
            else :
                expr.append(and_oprand[i])

#Instance Variables:
#   stmt, out_var, args, expr

    def __init__(self,stmt):
        self.stmt = stmt
        self.args = defaultdict(list)
        self.expr = []
        self.parse()
        return
    
    def __str__(self):
        return 'Statement : %s\nOutput Variable : %s\nArguments : %s\nExpression : %s' % (self.stmt, self.out_var, self.args, self.expr)
    
    def __repr__(self):
        return '\n%s\n' % (self.__str__())

    def ands_expr(self,axioms):
        and_oprand = []
        while axioms :
            axiom = axioms.pop()
            if axiom == '&' :
                self.expr.append(axiom)
                Rule.append_oprands(self.expr,and_oprand)
                and_oprand.clear()
            else :
                and_oprand.append(axiom)
                #put output Variables in the rule arguments
                if axiom != '^' :
                    arg = Rule.get_arg(axiom)
                    self.args[arg[0]].append(arg[1])
        Rule.append_oprands(self.expr,and_oprand)
        return

    def or_expr(self,axioms):
        or_oprand = []
        while axioms :
            axiom = axioms.pop()
            if axiom == '|' :
                self.expr.append(axiom)
                self.ands_expr(or_oprand)
                or_oprand.clear()
            else :
                or_oprand.insert(0,axiom)
        self.ands_expr(or_oprand)
        return

    def parse(self):
        axioms = self.stmt.split(' ')
        self.out_var = Rule.get_arg(axioms.pop())
        #pop the (->)
        axioms.pop()   
        #Put The Whole Expression
        self.or_expr(axioms)
        return
    
    def evaluate(self,mus):
        expr = self.expr.copy() 
        mu2 = None
        while expr:
            axiom = expr.pop()
            if axiom == '^' :
                mu2 = Rule.NOT(mu2)
            elif axiom == '&' :
                mu2 = Rule.AND(mu1,mu2)
            elif axiom == '|' :
                mu2 = Rule.OR(mu1,mu2)
            else :
                mu1 = mu2
                mu2 = mus[axiom]
        return self.out_var , mu2
    

In [ ]:
Unit of INFERENCING

In [509]:
mus = {
        ('z','g') : 0.75, ('z','f') : 0.3, ('z','e') : 0.1,
        ('y','d') : 0.5, ('y','c') : 0.2,
        ('x','b') : 0.2, ('x','a') : 0.1
      }
mus3 = {
        ('x','b') : 0.2, ('x','a') : 0.1, ('x','c') : 0.3
       }
r1 = Rule('x=a & x=b | y=c & y=d | z=e & z=f & z=g -> m=n')
r2 = Rule('x=a & ^ x=b | y=c & y=d | z=e & ^ z=f & z=g -> m=n')
r3 = Rule('x=c & x=b | x=a -> m=n')
print([r1,r2])
print(r1.evaluate(mus))
print(r3.evaluate(mus3))


[
Statement : x=a & x=b | y=c & y=d | z=e & z=f & z=g -> m=n
Output Variable : ('m', 'n')
Arguments : defaultdict(<class 'list'>, {'z': ['g', 'f', 'e'], 'y': ['d', 'c'], 'x': ['b', 'a']})
Expression : ['|', '&', ('z', 'g'), '&', ('z', 'f'), ('z', 'e'), '|', '&', ('y', 'd'), ('y', 'c'), '&', ('x', 'b'), ('x', 'a')]
, 
Statement : x=a & ^ x=b | y=c & y=d | z=e & ^ z=f & z=g -> m=n
Output Variable : ('m', 'n')
Arguments : defaultdict(<class 'list'>, {'z': ['g', 'f', 'e'], 'y': ['d', 'c'], 'x': ['b', 'a']})
Expression : ['|', '&', ('z', 'g'), '&', '^', ('z', 'f'), ('z', 'e'), '|', '&', ('y', 'd'), ('y', 'c'), '&', '^', ('x', 'b'), ('x', 'a')]
]
(('m', 'n'), 0.1)
(('m', 'n'), 0.2)


In [510]:
class FuzzyControlKit:

#Instance Variables:
#   prev_var, prev_type, in_vars, out_vars, rules 

    def __init__(self):
        self.prev_var  = None
        self.prev_type = None
        self.in_vars = {}
        self.out_vars = {}
        self.rules = []
        return
    
    def __str__(self):
        return 'Name of Last Variable Added : %s\nType of Last Variable Added : %s\nInput Variables :\n%s\nOutput Variables :\n%s\nRules :\n%s' % (self.prev_var, self.prev_type, self.in_vars, self.out_vars, self.rules)
    
    def __repr__(self):
        return self.__str__()

    def add_var(self,name,type):
        self.prev_var  = name
        self.prev_type = type
        if type.value == Variable.Type.input.value :
            self.in_vars[name]  = Variable()  
        else :
            self.out_vars[name] = Variable()
        return
        
    def set_var_set(self,name,x):
        if self.prev_type.value == Variable.Type.input.value :
            self.in_vars[self.prev_var].update_set(name,x)
        else :
            self.out_vars[self.prev_var].update_set(name,x)
        return
    
    def set_var_crisp(self,name,x):
        self.in_vars[name].set_crisp(x)
        return
    
    def add_rule(self,rule):
        self.rules.append(Rule(rule))
        return

    def input_fuzzification(self):
        print('Input Variables\' Sets Fuzzification Result :')
        for var in self.in_vars :
            print(var,':')
            print(self.in_vars[var],'\n')
        return

    def get_args_mus(self,args):
        args_mus = {}
        for var in args :
            for st in args[var] :
                args_mus[(var,st)] = self.in_vars[var].get_set_mu(st)
        return args_mus

    def inference(self):
        out_var_set_mus = {}
        for rule in self.rules :
            args_mus = self.get_args_mus(rule.args)
            print(args_mus)
            var_set_of_currmu , mu = rule.evaluate(args_mus)
            if var_set_of_currmu[0] not in out_var_set_mus :
                out_var_set_mus[var_set_of_currmu[0]] = defaultdict(lambda: -1)
            out_var_set_mus[var_set_of_currmu[0]][var_set_of_currmu[1]] = Rule.OR(mu,              out_var_set_mus[var_set_of_currmu[0]][var_set_of_currmu[1]])
        print(out_var_set_mus)
        return out_var_set_mus
    
    #Weighted Average
    def output_defuzzification(self,out_var_set_mus):
        out_var_values = []
        for var in out_var_set_mus :
            numrtr = 0
            dnmrtr = 0
            for var_set in out_var_set_mus[var] :
                numrtr += self.out_vars[var].get_set_centriod(var_set) * out_var_set_mus[var][var_set]
                dnmrtr += out_var_set_mus[var][var_set]
                z = numrtr / dnmrtr
            out_var_values.append((var, z, self.out_vars[var].get_likelhood_set(z)))
        return out_var_values

    

In [517]:
kit = FuzzyControlKit()
kit.add_var('Project_Funding',Variable.Type.input)
kit.set_var_set('vl',(0,0,10,30))
kit.set_var_set('l',(10,30,40,60))
kit.set_var_set('m',(40,60,70,90))
kit.set_var_set('h',(70,90,100,100))

In [518]:
kit.add_var('Team_Experience_Level',Variable.Type.input)
kit.set_var_set('b',(0,15,30))
kit.set_var_set('i',(15,30,45))
kit.set_var_set('e',(30,60,60))

In [519]:
kit.add_var('Risk',Variable.Type.output)
kit.set_var_set('l',(0,25,50))
kit.set_var_set('n',(25,50,75))
kit.set_var_set('h',(50,100,100))

In [520]:
kit.add_rule('Project_Funding=h | Team_Experience_Level=e -> Risk=l')
kit.add_rule('Project_Funding=m & Team_Experience_Level=i | Team_Experience_Level=b -> Risk=n')
kit.add_rule('Project_Funding=l -> Risk=h')
kit.add_rule('Project_Funding=l & Team_Experience_Level=b -> Risk=h')

In [521]:
kit.set_var_crisp('Project_Funding',50)
kit.set_var_crisp('Team_Experience_Level',40)
kit.input_fuzzification()
out_var_set_mus = kit.inference()
kit.output_defuzzification(out_var_set_mus)

Input Variables' Sets Fuzzification Result :
Project_Funding :
Crisp Value : 50
Sets :{'vl': 
Type : 2
Vertices Number : 4
X Vertices : (0, 0, 10, 30)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.0, 1.0), (-0.05, 1.5)]
There Intervals : [(0, 10), (10, 30)]
Centriod : 10.833333333333334
, 'l': 
Type : 2
Vertices Number : 4
X Vertices : (10, 30, 40, 60)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -0.5), (0.0, 1.0), (-0.05, 3.0)]
There Intervals : [(10, 30), (30, 40), (40, 60)]
Centriod : 35.0
, 'm': 
Type : 2
Vertices Number : 4
X Vertices : (40, 60, 70, 90)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -2.0), (0.0, 1.0), (-0.05, 4.5)]
There Intervals : [(40, 60), (60, 70), (70, 90)]
Centriod : 65.0
, 'h': 
Type : 2
Vertices Number : 4
X Vertices : (70, 90, 100, 100)
Y Vertices : (0, 1, 1, 0)
Line Equations :  [(0.05, -3.5), (0.0, 1.0)]
There Intervals : [(70, 90), (90, 100)]
Centriod : 89.16666666666667
} 

Team_Experience_Level :
Crisp Value : 40
Sets :{'b': 
Type : 1


KeyError: 'l'

In [ ]:
out_var_set_vals = {}
out_var_set_vals['k'] = defaultdict(lambda: -1)
out_var_set_vals['k']['l'] = 5
if 'c' not in out_var_set_vals :
    out_var_set_vals['c'] = defaultdict(lambda: -1)
out_var_set_vals['c']['n'] = 20
out_var_set_vals
